# Chapter 3: Coding Attention Mechanisms

## 3.3 Attending to different parts of the input with self-attention

### 3.3.1 A simple self-attention mechanism without trainable weights

In [2]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your      x^1
     [0.55, 0.87, 0.66], # journey   x^2
     [0.57, 0.85, 0.64], # starts    x^3
     [0.22, 0.58, 0.33], # with      x^4
     [0.77, 0.25, 0.10], # one       x^5
     [0.05, 0.80, 0.55]] # step      x^6
)

In [3]:
input_query = inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [4]:
input_1 = inputs[0]
input_1

tensor([0.4300, 0.1500, 0.8900])

In [5]:
torch.dot(input_query, input_1)

tensor(0.9544)

In [7]:
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [9]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
attn_weights_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [11]:
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i

context_vec_2

tensor([0.4419, 0.6515, 0.5683])